Importing libraries and reading the datasets

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, ConnectionPatch, Arc
from matplotlib.font_manager import FontProperties
from functools import reduce
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

df21 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_21.csv')
df20 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_20.csv')
df19 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_19.csv')
df18 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_18.csv')
df17 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_17.csv')
df16 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_16.csv')
df15 = pd.read_csv('/kaggle/input/fifa-21-complete-player-dataset/players_15.csv')

Average rating by age comparison between the FIFA versions

In [ ]:
#from the kernel https://www.kaggle.com/laowingkin/fifa-18-find-the-best-squad
#thanks DLao

df21_pot = df21[(df21.age.astype(int) >= 18) & (df21.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df21_over = df21[(df21.age.astype(int) >= 18) & (df21.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df21_summary = pd.concat([df21_pot, df21_over], axis=1)
df20_pot = df20[(df20.age.astype(int) >= 18) & (df20.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df20_over = df20[(df20.age.astype(int) >= 18) & (df20.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df20_summary = pd.concat([df20_pot, df20_over], axis=1)
df19_pot = df19[(df19.age.astype(int) >= 18) & (df19.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df19_over = df19[(df19.age.astype(int) >= 18) & (df19.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df19_summary = pd.concat([df19_pot, df19_over], axis=1)
df18_pot = df18[(df18.age.astype(int) >= 18) & (df18.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df18_over = df18[(df18.age.astype(int) >= 18) & (df18.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df18_summary = pd.concat([df18_pot, df18_over], axis=1)
df17_pot = df17[(df17.age.astype(int) >= 18) & (df17.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df17_over = df17[(df17.age.astype(int) >= 18) & (df17.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df17_summary = pd.concat([df17_pot, df17_over], axis=1)
df16_pot = df16[(df16.age.astype(int) >= 18) & (df16.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df16_over = df16[(df16.age.astype(int) >= 18) & (df16.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df16_summary = pd.concat([df16_pot, df16_over], axis=1)
df15_pot = df15[(df15.age.astype(int) >= 18) & (df15.age.astype(int) <= 35)].groupby(['age'])['potential'].mean()
df15_over = df15[(df15.age.astype(int) >= 18) & (df15.age.astype(int) <= 35)].groupby(['age'])['overall'].mean()
df15_summary = pd.concat([df15_pot, df15_over], axis=1)

#from the kernel https://www.kaggle.com/laowingkin/fifa-18-find-the-best-squad
#thanks DLao
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(nrows=3, ncols=2, figsize=(25, 20))
ax1.plot(df21_summary)
ax1.set_xlabel('Age')
ax1.set_ylabel('Rating')
ax1.set_title('FIFA 21 - Average Rating by Age')
ax2.plot(df20_summary)
ax2.set_xlabel('Age')
ax2.set_ylabel('Rating')
ax2.set_title('FIFA 20 - Average Rating by Age')
ax3.plot(df19_summary)
ax3.set_xlabel('Age')
ax3.set_ylabel('Rating')
ax3.set_title('FIFA 19 - Average Rating by Age')
ax4.plot(df18_summary)
ax4.set_xlabel('Age')
ax4.set_ylabel('Rating')
ax4.set_title('FIFA 18 - Average Rating by Age')
ax5.plot(df17_summary)
ax5.set_xlabel('Age')
ax5.set_ylabel('Rating')
ax5.set_title('FIFA 17 - Average Rating by Age')
ax6.set_xlabel('Age')
ax6.plot(df16_summary)
ax6.set_ylabel('Rating')
ax6.set_title('FIFA 16 - Average Rating by Age')

Dream Teams from FIFA 16 to 21 to play with 4-3-3 and 3-5-2

In [ ]:
# adding the 'best_position' and 'value_million_eur' fields to each df
def add_position_and_value_fields(input_df):
    input_df['best_position'] = input_df['player_positions'].str.split(',').str[0]
    input_df['value_million_eur'] = pd.to_numeric(input_df['value_eur'], errors='coerce') / 1000000
    return input_df

df21 = add_position_and_value_fields(df21)
df20 = add_position_and_value_fields(df20)
df19 = add_position_and_value_fields(df19)
df18 = add_position_and_value_fields(df18)
df17 = add_position_and_value_fields(df17)
df16 = add_position_and_value_fields(df16)
df15 = add_position_and_value_fields(df15)

In [ ]:
df21.head(5)

Defining the functions to retrieve the best squad for each club and then draw the lineup given the chosen formation

In [ ]:
# original code taken from https://towardsdatascience.com/advanced-sports-visualization-with-pandas-matplotlib-and-seaborn-9c16df80a81b

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, ConnectionPatch, Arc

# 4-3-3
squad_433 = ['GK', 'LB|LWB|LDM', 'LCB|CB', 'RCB|CB', 'RB|RWB|RDM', 'CDM|CM|CAM', 'CDM|CM|CAM', 'CDM|CM|CAM', 'LF|LW|CF|ST', 'CF|ST', 'RF|RW|CF|ST']
# 3-5-2
squad_352 = ['GK', 'LB|LCB|CB', 'CB', 'RB|RCB|CB', 'LW|LWB|LM|LCM', 'CDM|CM', 'CDM|CM', 'CM|CAM|CF', 'RW|RWB|RM|RCM', 'LC|LW|CF|ST', 'RC|RW|CF|ST']


def get_best_lineup(df_name, squad_lineup, club_name='', measurement='overall'):
    df_copy = df_name.copy()
    if club_name != '':
        df_copy = df_copy[df_copy['club_name'] == club_name]
    squad_default_dict = dict()
    for i in squad_lineup:
        squad_default_dict[df_copy.loc[[df_copy[df_copy['best_position'].str.contains(i)][measurement].idxmax()]]['short_name'].to_string(index = False)] = [
            df_copy.loc[[df_copy[df_copy['best_position'].str.contains(i)][measurement].idxmax()]]['best_position'].to_string(index = False),
            df_copy[df_copy['best_position'].str.contains(i)][measurement].max(),
            int(df_copy.loc[[df_copy[df_copy['best_position'].str.contains(i)][measurement].idxmax()]]['age'].to_string(index = False)),
            float(df_copy.loc[[df_copy[df_copy['best_position'].str.contains(i)][measurement].idxmax()]]['value_million_eur'].to_string(index = False)),
            df_copy.loc[[df_copy[df_copy['best_position'].str.contains(i)][measurement].idxmax()]]['club_name'].to_string(index = False)]
        df_copy.drop(df_copy[df_copy['best_position'].str.contains(i)][measurement].idxmax(), inplace = True)
    return squad_default_dict


def draw_pitch(ax):
    # pitch outline and centre line
    pitch = Rectangle([0, 0], width=80, height=120, edgecolor='black', fill=False) # facecolor='#23E04F'
    # left and right penalty area and midline
    left_penalty = Rectangle([22.3, 0], width=35.3, height=14.6, fill=False)
    right_penalty = Rectangle([22.3, 105.4], width=35.3, height=14.6, fill=False)
    midline = ConnectionPatch([0, 60], [80, 60], 'data', 'data')
    # left and right six-yard box
    left_six_yard = Rectangle([32, 0], width=16, height=4.9, fill=False)
    right_six_yard = Rectangle([32, 115.1], width=16, height=4.9, fill=False)
    # prepare circles
    centre_circle = plt.Circle((40, 60), 8.1, color='black', fill=False)
    centre_spot = plt.Circle((40, 60), 0.4, color='black')
    # penalty spots and arcs around penalty boxes
    # left_pen_spot = plt.Circle((40, 9.7), 0.4, color='black')
    # right_pen_spot = plt.Circle((40, 110.3), 0.4, color='black')
    left_arch = Arc((40, 9.5), width=16.2, height=16.2, angle=90, theta1=310, theta2=50, color='black')
    right_arch = Arc((40, 110.4), width=16.2, height=16.2, angle=90, theta1=130, theta2=230, color='black')
    elements_list = [pitch, left_penalty, right_penalty, midline, left_six_yard, right_six_yard, centre_circle,
                     centre_spot, left_arch, right_arch]
    for element in elements_list:
        ax.add_patch(element)


def draw_players(home_team_dictionary, home_team_name, home_team_module, away_team_dictionary, away_team_name, away_team_module):
    note_columns = ('Position', 'Player Name', 'Overall Attribute', 'Age', 'Player Value (in €M)', 'Club Name')
    # home team lineup drawn
    home_players_list = list(home_team_dictionary)
    if home_team_module == '4-3-3':
        for i in range(len(home_players_list)):
            # setting color based on generic role - goalkeeper, defender, midfielder, striker
            if i == 0:
                circle_color = 'grey'
            elif 1 <= i <= 4:
                circle_color = 'blue'
            elif 5 <= i <= 7:
                circle_color = 'gold'
            elif 8 <= i <= 10:
                circle_color = 'red'
            # setting XY coordinates based on specific role - GK, LB, CB, etc.
            if i == 0:
                player_x, player_y = 40, 112
            elif i == 1:
                player_x, player_y = 70, 98
            elif i == 2:
                player_x, player_y = 50, 98
            elif i == 3:
                player_x, player_y = 30, 98
            elif i == 4:
                player_x, player_y = 10, 98
            elif i == 5:
                player_x, player_y = 55, 84
            elif i == 6:
                player_x, player_y = 40, 84
            elif i == 7:
                player_x, player_y = 25, 84
            elif i == 8:
                player_x, player_y = 55, 70
            elif i == 9:
                player_x, player_y = 40, 70
            elif i == 10:
                player_x, player_y = 25, 70
            if '. ' in home_players_list[i]:
                player_name = home_players_list[i].split('. ', 1)[1]
            else:
                player_name = home_players_list[i]
            plt.annotate(player_name,
                         xy = (player_x, player_y), xytext = (0, 18),
                         bbox=dict(boxstyle="round", fc="w"),
                         va="center", ha="center", textcoords='offset points')
            plt.scatter(player_x, player_y, s=250, c=circle_color)
    elif home_team_module == '3-5-2':
        for i in range(len(home_players_list)):
            # setting color based on generic role - goalkeeper, defender, midfielder, striker
            if i == 0:
                circle_color = 'grey'
            elif 1 <= i <= 3:
                circle_color = 'blue'
            elif 4 <= i <= 8:
                circle_color = 'gold'
            elif 9 <= i <= 10:
                circle_color = 'red'
            # setting XY coordinates based on specific role - GK, LB, CB, etc.
            if i == 0:
                player_x, player_y = 40, 112
            elif i == 1:
                player_x, player_y = 55, 98
            elif i == 2:
                player_x, player_y = 40, 98
            elif i == 3:
                player_x, player_y = 25, 98
            elif i == 4:
                player_x, player_y = 74, 84
            elif i == 5:
                player_x, player_y = 57, 84
            elif i == 6:
                player_x, player_y = 40, 84
            elif i == 7:
                player_x, player_y = 23, 84
            elif i == 8:
                player_x, player_y = 6, 84
            elif i == 9:
                player_x, player_y = 50, 70
            elif i == 10:
                player_x, player_y = 30, 70
            if '. ' in home_players_list[i]:
                player_name = home_players_list[i].split('. ', 1)[1]
            else:
                player_name = home_players_list[i]
            plt.annotate(player_name,
                         xy = (player_x, player_y), xytext = (0, 18),
                         bbox=dict(boxstyle="round", fc="w"),
                         va="center", ha="center", textcoords='offset points')
            plt.scatter(player_x, player_y, s=250, c=circle_color)
    else:
        print('module name ' + home_team_module + ' is not valid')
    # adding notes on the right-hand side of the home team
    home_team_list = []
    for k, v in home_team_dictionary.items():
        home_team_list.append([v[0], k, v[1], v[2], v[3], v[4]])
    home_sum_rating = 0
    home_sum_age = 0
    home_sum_value = 0
    for k, v in home_team_dictionary.items():
        home_sum_rating = home_sum_rating + v[1]
        home_sum_age = home_sum_age + v[2]
        home_sum_value = home_sum_value + v[3]
    home_notes = [[home_team_name],
                  ['Average rating: {avg_rating}'.format(avg_rating=round((home_sum_rating/11), 1))],
                  ['Average age: {avg_age}'.format(avg_age=round((home_sum_age/11), 1))],
                  ['Total Value (in €M): {total_value}'.format(total_value=round(home_sum_value, 1))]]
    plt_table = plt.table(cellText=home_team_list, colLabels=note_columns,
                          colWidths=[0.3, 0.5, 0.35, 0.2, 0.4, 0.5],
                          cellLoc='right', loc='right',
                          bbox=[1, 0.505, 1.7, 0.36])
    plt_table.scale(1.5, 2)
    for (row, col), cell in plt_table.get_celld().items():
        if (row == 0):
            cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    plt_home_notes = plt.table(cellText=home_notes,
                               cellLoc='left', loc='left',
                               bbox=[0.9, 0.87, 1.1, 0.12])
    for key, cell in plt_home_notes.get_celld().items():
        cell.set_linewidth(0)
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    # away team lineup drawn
    away_players_list = list(away_team_dictionary)
    if away_team_module == '4-3-3':
        for i in range(len(away_players_list)):
            # setting color based on generic role - goalkeeper, defender, midfielder, striker
            if i == 0:
                circle_color = 'grey'
            elif 1 <= i <= 4:
                circle_color = 'blue'
            elif 5 <= i <= 7:
                circle_color = 'gold'
            elif 8 <= i <= 10:
                circle_color = 'red'
            # setting XY coordinates based on specific role - GK, LB, CB, etc.
            if i == 0:
                player_x, player_y = 40, 8
            elif i == 1:
                player_x, player_y = 10, 22
            elif i == 2:
                player_x, player_y = 30, 22
            elif i == 3:
                player_x, player_y = 50, 22
            elif i == 4:
                player_x, player_y = 70, 22
            elif i == 5:
                player_x, player_y = 25, 36
            elif i == 6:
                player_x, player_y = 40, 36
            elif i == 7:
                player_x, player_y = 55, 36
            elif i == 8:
                player_x, player_y = 25, 50
            elif i == 9:
                player_x, player_y = 40, 50
            elif i == 10:
                player_x, player_y = 55, 50
            if '. ' in away_players_list[i]:
                player_name = away_players_list[i].split('. ', 1)[1]
            else:
                player_name = away_players_list[i]
            plt.annotate(player_name,
                         xy = (player_x, player_y), xytext = (0, -18),
                         bbox=dict(boxstyle="round", fc="w"),
                         va="center", ha="center", textcoords='offset points')
            plt.scatter(player_x, player_y, s=250, c=circle_color)
    elif away_team_module == '3-5-2':
        for i in range(len(away_players_list)):
            # setting color based on generic role - goalkeeper, defender, midfielder, striker
            if i == 0:
                circle_color = 'grey'
            elif 1 <= i <= 3:
                circle_color = 'blue'
            elif 4 <= i <= 8:
                circle_color = 'gold'
            elif 9 <= i <= 10:
                circle_color = 'red'
            # setting XY coordinates based on specific role - GK, LB, CB, etc.
            if i == 0:
                player_x, player_y = 40, 8
            elif i == 1:
                player_x, player_y = 25, 22
            elif i == 2:
                player_x, player_y = 40, 22
            elif i == 3:
                player_x, player_y = 55, 22
            elif i == 4:
                player_x, player_y = 6, 36
            elif i == 5:
                player_x, player_y = 23, 36
            elif i == 6:
                player_x, player_y = 40, 36
            elif i == 7:
                player_x, player_y = 57, 36
            elif i == 8:
                player_x, player_y = 74, 36
            elif i == 9:
                player_x, player_y = 30, 50
            elif i == 10:
                player_x, player_y = 50, 50
            if '. ' in away_players_list[i]:
                player_name = away_players_list[i].split('. ', 1)[1]
            else:
                player_name = away_players_list[i]
            plt.annotate(player_name,
                         xy = (player_x, player_y), xytext = (0, -18),
                         bbox=dict(boxstyle="round", fc="w"),
                         va="center", ha="center", textcoords='offset points')
            plt.scatter(player_x, player_y, s=250, c=circle_color)
    else:
        print('module name ' + away_team_module + ' is not valid')
    # adding notes on the right-hand side of the away team
    away_team_list = []
    for k, v in away_team_dictionary.items():
        away_team_list.append([v[0], k, v[1], v[2], v[3], v[4]])
    away_sum_rating = 0
    away_sum_age = 0
    away_sum_value = 0
    for k, v in away_team_dictionary.items():
        away_sum_rating = away_sum_rating + v[1]
        away_sum_age = away_sum_age + v[2]
        away_sum_value = away_sum_value + v[3]
    away_notes = [[away_team_name],
                  ['Average rating: {avg_rating}'.format(avg_rating=round((away_sum_rating/11), 1))],
                  ['Average age: {avg_age}'.format(avg_age=round((away_sum_age/11), 1))],
                  ['Total Value (in €M): {total_value}'.format(total_value=round(away_sum_value, 1))]]
    plt_table = plt.table(cellText=away_team_list, colLabels=note_columns,
                          colWidths=[0.3, 0.5, 0.35, 0.2, 0.4, 0.5],
                          cellLoc='right', loc='right',
                          bbox=[1, 0.015, 1.7, 0.36])
    plt_table.scale(1.5, 2)
    for (row, col), cell in plt_table.get_celld().items():
        if (row == 0):
            cell.set_text_props(fontproperties=FontProperties(weight='bold'))
    plt_away_notes = plt.table(cellText=away_notes,
                               cellLoc='left', loc='left',
                               bbox=[0.9, 0.38, 1.1, 0.12])
    for key, cell in plt_away_notes.get_celld().items():
        cell.set_linewidth(0)
        cell.set_text_props(fontproperties=FontProperties(weight='bold'))

Dream Team of FIFA 21 vs FIFA 15 with 4-3-3 and 3-5-2 lineups (Overall attributes)

In [ ]:
best_players_21_433 = get_best_lineup(df21, squad_433)
best_players_15_433 = get_best_lineup(df15, squad_433)

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(best_players_21_433, '4-3-3 Dream Team in FIFA 21', '4-3-3', best_players_15_433, '4-3-3 Dream Team in FIFA 15', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
best_players_21_352 = get_best_lineup(df21, squad_352)
best_players_15_352 = get_best_lineup(df15, squad_352)

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(best_players_21_352, '3-5-2 Dream Team in FIFA 21', '3-5-2', best_players_15_352, '3-5-2 Dream Team in FIFA 15', '3-5-2')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Dream Team of FIFA 21 vs FIFA 15 with 4-3-3 and 3-5-2 lineups (Potential attributes)

In [ ]:
best_players_21_433 = get_best_lineup(df21, squad_433, measurement='potential')
best_players_15_433 = get_best_lineup(df15, squad_433, measurement='potential')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(best_players_21_433, '4-3-3 Best Potential Team in FIFA 21', '4-3-3', best_players_15_433, '4-3-3 Best Potential Team in FIFA 15', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
best_players_21_352 = get_best_lineup(df21, squad_352, measurement='potential')
best_players_15_352 = get_best_lineup(df15, squad_352, measurement='potential')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(best_players_21_352, '3-5-2 Best Potential Team in FIFA 21', '3-5-2', best_players_15_352, '3-5-2 Best Potential Team in FIFA 15', '3-5-2')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Liverpool and ManCity FIFA 21 and FIFA 15 squads with the best players (Overall attributes)

In [ ]:
liverpool_21_433 = get_best_lineup(df21, squad_433, club_name='Liverpool')
mancity_21_433 = get_best_lineup(df21, squad_433, club_name='Manchester City')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(liverpool_21_433, '4-3-3 Liverpool in FIFA 21', '4-3-3', mancity_21_433, '4-3-3 Man City in FIFA 21', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
liverpool_15_433 = get_best_lineup(df15, squad_433, club_name='Liverpool')
mancity_15_433 = get_best_lineup(df15, squad_433, club_name='Manchester City')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(liverpool_15_433, '4-3-3 Liverpool in FIFA 15', '4-3-3', mancity_15_433, '4-3-3 Man City in FIFA 15', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Real Madrid and Barcelona FIFA 21 and FIFA 15 squads with the best players (Overall attributes)

In [ ]:
realmadrid_21_433 = get_best_lineup(df21, squad_433, club_name='Real Madrid')
barcelona_21_433 = get_best_lineup(df21, squad_433, club_name='FC Barcelona')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(realmadrid_21_433, '4-3-3 Real Madrid in FIFA 21', '4-3-3', barcelona_21_433, '4-3-3 Barcelona in FIFA 21', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
realmadrid_15_433 = get_best_lineup(df15, squad_433, club_name='Real Madrid')
barcelona_15_433 = get_best_lineup(df15, squad_433, club_name='FC Barcelona')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(realmadrid_15_433, '4-3-3 Real Madrid in FIFA 15', '4-3-3', barcelona_15_433, '4-3-3 Barcelona in FIFA 15', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Juventus and Inter FIFA 21 and FIFA 15 squads with the best players (Overall attributes)

In [ ]:
juventus_21_433 = get_best_lineup(df21, squad_433, club_name='Juventus')
inter_21_352 = get_best_lineup(df21, squad_352, club_name='Inter')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(juventus_21_433, '4-3-3 Juventus in FIFA 21', '4-3-3', inter_21_352, '3-5-2 Inter in FIFA 21', '3-5-2')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
juventus_15_352 = get_best_lineup(df15, squad_352, club_name='Juventus')
inter_15_352 = get_best_lineup(df15, squad_352, club_name='Inter')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(juventus_15_352, '3-5-2 Juventus in FIFA 15', '3-5-2', inter_15_352, '3-5-2 Inter in FIFA 15', '3-5-2')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Bayern Munich and Borussia Dortmund FIFA 21 and FIFA 15 squads with the best players (Overall attributes)

In [ ]:
bayern_21_352 = get_best_lineup(df21, squad_352, club_name='FC Bayern München')
bvb_21_352 = get_best_lineup(df21, squad_352, club_name='Borussia Dortmund')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(bayern_21_352, '4-3-3 Bayern Munich in FIFA 21', '3-5-2', bvb_21_352, '4-3-3 Borussia Dortmund in FIFA 21', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
bayern_15_352 = get_best_lineup(df15, squad_352, club_name='FC Bayern München')
bvb_15_352 = get_best_lineup(df15, squad_352, club_name='Borussia Dortmund')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(bayern_15_352, '3-5-2 Bayern Munich in FIFA 15', '3-5-2', bvb_15_352, '3-5-2 Borussia Dortmund in FIFA 15', '3-5-2')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Paris Saint-Germain and Olympique Lyonnais FIFA 21 and FIFA 15 squads with the best players (Overall attributes)

In [ ]:
psg_21_352 = get_best_lineup(df21, squad_352, club_name='Paris Saint-Germain')
lyon_21_433 = get_best_lineup(df21, squad_433, club_name='Olympique Lyonnais')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(psg_21_352, '3-5-2 Paris Saint-Germain in FIFA 21', '3-5-2', lyon_21_433, '4-3-3 Olympique Lyonnais in FIFA 21', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

In [ ]:
psg_15_352 = get_best_lineup(df15, squad_352, club_name='Paris Saint-Germain')
lyon_15_433 = get_best_lineup(df15, squad_433, club_name='Olympique Lyonnais')

fig = plt.figure()
fig.set_size_inches(10, 14)
ax = fig.add_subplot(1, 1, 1)
draw_pitch(ax)
draw_players(psg_15_352, '3-5-2 Paris Saint-Germain in FIFA 15', '3-5-2', lyon_15_433, '4-3-3 Olympique Lyonnais in FIFA 15', '4-3-3')
plt.xlim(-2, 82)
plt.ylim(-2, 122)
plt.axis('off')
plt.show()

Top 11 FIFA players from the version 15 to FIFA 21

In [ ]:
def get_top10_players(df_name):
    df_copy = df_name.sort_values(['overall', 'potential'], ascending=[False, False]).head(10)
    store = []
    for index, row in df_copy.iterrows():
        store.append([row['best_position'], row['short_name'], row['overall'], row['potential'], row['age']])
    return np.mean([x[2] for x in store]).round(1), np.mean([x[3] for x in store]).round(1), np.mean([x[4] for x in store]).round(1), pd.DataFrame(np.array(store).reshape(10, 5), columns = ['Position', 'Player', 'Overall', 'Potential', 'Age']).to_string(index = False)

top10_players21_overall, top10_players21_potential, top10_players21_age, top10_players21 = get_top10_players(df21)
print('FIFA 21 - Top 10 Players')
print('Average overall: {:.1f}'.format(top10_players21_overall))
print('Average potential: {:.1f}'.format(top10_players21_potential))
print('Average age: {:.1f}'.format(top10_players21_age))
print(top10_players21)

top10_players20_overall, top10_players20_potential, top10_players20_age, top10_players20 = get_top10_players(df20)
print('\nFIFA 20 - Top 10 Players')
print('Average overall: {:.1f}'.format(top10_players20_overall))
print('Average potential: {:.1f}'.format(top10_players20_potential))
print('Average age: {:.1f}'.format(top10_players20_age))
print(top10_players20)

top10_players19_overall, top10_players19_potential, top10_players19_age, top10_players19 = get_top10_players(df19)
print('\nFIFA 19 - Top 10 Players')
print('Average overall: {:.1f}'.format(top10_players19_overall))
print('Average potential: {:.1f}'.format(top10_players19_potential))
print('Average age: {:.1f}'.format(top10_players19_age))
print(top10_players19)

top10_players18_overall, top10_players18_potential, top10_players18_age, top10_players18 = get_top10_players(df18)
print('\nFIFA 18 - Top 10 Players')
print('Average overall: {:.1f}'.format(top10_players18_overall))
print('Average potential: {:.1f}'.format(top10_players18_potential))
print('Average age: {:.1f}'.format(top10_players18_age))
print(top10_players18)

top10_players17_overall, top10_players17_potential, top10_players17_age, top10_players17 = get_top10_players(df17)
print('\nFIFA 17 - Top 10 Players')
print('Average overall: {:.1f}'.format(top10_players17_overall))
print('Average potential: {:.1f}'.format(top10_players17_potential))
print('Average age: {:.1f}'.format(top10_players17_age))
print(top10_players17)

top10_players16_overall, top10_players16_potential, top10_players16_age, top10_players16 = get_top10_players(df16)
print('\nFIFA 16 - Top 10 Players')
print('Average overall: {:.1f}'.format(top10_players16_overall))
print('Average potential: {:.1f}'.format(top10_players16_potential))
print('Average age: {:.1f}'.format(top10_players16_age))
print(top10_players16)

In [ ]:
top_players21_list = df21.loc[(df21['overall'] >= 90)].sort_values(['overall', 'potential'], ascending=[False, False])['long_name'].tolist()

# Mbappé will not be included in the chart as was not available in FIFA 15 and 16
['Lionel Andrés Messi Cuccittini',
 'Cristiano Ronaldo dos Santos Aveiro',
 'Jan Oblak',
 'Robert Lewandowski',
 'Neymar da Silva Santos Júnior',
 'Kevin De Bruyne',
 'Kylian Mbappé Lottin',
 'Marc-André ter Stegen',
 'Virgil van Dijk',
 'Alisson Ramsés Becker',
 'Sadio Mané',
 'Mohamed Salah Ghaly']

# manually amending the Neymar short_name on FIFA 20-21
df20.loc[df20['short_name'] == 'Neymar Jr', 'short_name'] = 'Neymar'
df21.loc[df21['short_name'] == 'Neymar Jr', 'short_name'] = 'Neymar'

#VVD was not included in the sofifa database as of 2015-09-21 (not anymore in the Celtic's squad, not yet in the Southamption's squad)
if df16.index.max() == 15622:
    df16.loc[df16.index.max() + 1] = 203376
df16.loc[df16['sofifa_id'] == 203376, 'short_name'] = 'V. van Dijk'
df16.loc[df16['sofifa_id'] == 203376, 'long_name'] = 'Virgil van Dijk'
df16.loc[df16['sofifa_id'] == 203376, 'overall'] = 77
df16.loc[df16['sofifa_id'] == 203376, 'potential'] = 83

filtered_players21 = df21[df21['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players21.columns = ['short_name', 'overall21', 'potential21']
filtered_players21.set_index('short_name', inplace=True)
filtered_players20 = df20[df20['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players20.columns = ['short_name', 'overall20', 'potential20']
filtered_players20.set_index('short_name', inplace=True)
filtered_players19 = df19[df19['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players19.columns = ['short_name', 'overall19', 'potential19']
filtered_players19.set_index('short_name', inplace=True)
filtered_players18 = df18[df18['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players18.columns = ['short_name', 'overall18', 'potential18']
filtered_players18.set_index('short_name', inplace=True)
filtered_players17 = df17[df17['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players17.columns = ['short_name', 'overall17', 'potential17']
filtered_players17.set_index('short_name', inplace=True)
filtered_players16 = df16[df16['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players16.columns = ['short_name', 'overall16', 'potential16']
filtered_players16.set_index('short_name', inplace=True)
filtered_players15 = df15[df15['long_name'].isin(top_players21_list)][['short_name', 'overall', 'potential']]
filtered_players15.columns = ['short_name', 'overall15', 'potential15']
filtered_players15.set_index('short_name', inplace=True)

dfs = [filtered_players21, filtered_players20, filtered_players19, filtered_players18, filtered_players17, filtered_players16, filtered_players15]
merged_df = reduce(lambda left,right: pd.merge(left, right, on='short_name'), dfs)
overall_df = merged_df[['overall21', 'overall20', 'overall19', 'overall18', 'overall17', 'overall16', 'overall15']]
overall_df.columns = ['FIFA 21', 'FIFA 20', 'FIFA 19', 'FIFA 18', 'FIFA 17', 'FIFA 16', 'FIFA 15']
potential_df = merged_df[['potential21', 'potential20', 'potential19', 'potential18', 'potential17', 'potential16', 'potential15']]
potential_df.columns = ['FIFA 21', 'FIFA 20', 'FIFA 19', 'FIFA 18', 'FIFA 17', 'FIFA 16', 'FIFA 15']

Trend line of Top Players Overall Rating

In [ ]:
transposed_overall_df = overall_df.T.sort_index(ascending=True)
plt.rcParams["figure.figsize"] = [20, 12]
ax = transposed_overall_df.plot()
ax.set_ylabel('Overall Rating')
ax.set_title('FIFA 21 - Top Players Historical Overall Rating')

Trend line of Top Players Potential Rating

In [ ]:
transposed_potential_df = potential_df.T.sort_index(ascending=True)
plt.rcParams["figure.figsize"] = [20, 12]
ax = transposed_potential_df.plot()
ax.set_ylabel('Potential Rating')
ax.set_title('FIFA 21 - Top Players Historical Potential Rating')